# Azure AI Agent service - Custom Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingCustomSearchTool
from azure.ai.agents.models import ListSortOrder
from dotenv import load_dotenv

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 16-Jun-2025 14:07:19


## Settings

In [4]:
load_dotenv("azure.env")

True

In [5]:
BING_CUSTOM_CONN_ID = os.environ["BING_CUSTOM_CONN_ID"]

In [6]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

## Initialize Bing Custom Search tool with connection id and instance name

In [7]:
instance_name = "travel"

# Initialize Bing Custom Search tool with connection id and instance name
bing_custom_tool = BingCustomSearchTool(connection_id=BING_CUSTOM_CONN_ID,
                                        instance_name=instance_name)

Exemple of instance definition:

<img src="custom.jpg">

## Agent creation

In [8]:
agent = project_client.create_agent(
    model='gpt-4o',
    name="travelplanneragent",
    instructions="You are a travel planner",
    tools=bing_custom_tool.definitions,
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_oNmjwUSIbTIvrXMrdmFdkHPu


## Examples

In [12]:
prompt="List me the top 10 things to visit in Paris according to tripadvisor"

In [13]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_JHyOnZrbJCqP94fasf7qOMvv
💬 [2] Created message | ID: msg_XD180rxvL1daNAZ5hLT2l8l2
🏃 [3] Created run | ID: run_Qvupyefx5fvzrRGz9VYT9yaM
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → List me the top 10 things to visit in Paris according to tripadvisor

💡 → According to TripAdvisor, the following are the top attractions to visit in Paris:

1. **Eiffel Tower** - The iconic symbol of Paris offering stunning city views.
2. **Louvre Museum** - The world-renowned museum, home of the Mona Lisa.
3. **Cathédrale Notre-Dame de Paris** - A masterpiece of Gothic architecture, currently undergoing restoration.
4. **Sacré-Cœur Basilica** - Located on Montmartre Hill, it provides panoramic views of Paris.
5. **Musée d'Orsay** - Renowned for its collection of Impressionist masterpieces.
6. **Arc de Triomphe** - A historic monument offering a beautiful v

In [14]:
prompt="What are the best hotels in Paris according to social networks?"

In [15]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_0SPnknFshjzr0UYox30XtFQf
💬 [2] Created message | ID: msg_K81Lm4q6mpCojljs1WNuEhIb
🏃 [3] Created run | ID: run_w7DX4CbP9ZgoEDOUHF5A6ifc
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What are the best hotels in Paris according to social networks?

💡 → Some of the best and highly recommended hotels in Paris, often praised on social media and review platforms in 2025, include:

1. Hotel Le Bristol Paris - Known for combining Parisian elegance with modern luxury, it is a favorite for its dining and exceptional service.
2. Hôtel Plaza Athénée - A legendary spot famous for its glamorous location and design.
3. Shangri-La Hotel, Paris - Loved for its views of the Eiffel Tower and superior hospitality.
4. La Réserve Paris - Hotel and Spa - Valued by travelers for its intimate, luxurious setting.
5. Les Jardins Du Marais - Popular due to its cha

In [16]:
prompt = "Find some flights from Nice to Paris"

In [17]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_ptgYUWkoXylTnFzx39L3lvaE
💬 [2] Created message | ID: msg_c8Jo3CAU559BR81dzmjm28k8
🏃 [3] Created run | ID: run_aEZ8wH85w44S44FMWt7nm9OM
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → Find some flights from Nice to Paris

💡 → Several airlines operate flights from Nice to Paris, including Air France and easyJet. Direct flights from Nice Côte d'Azur Airport (NCE) to Paris are typically around 1.5 hours.

**Airfare Examples**: 
- EasyJet offers fares starting at €85 for a direct round trip.
- Air France provides flights starting from €129 with checked luggage【3:0†source】【3:2†source】.

For booking, consider platforms like KAYAK or directly with airlines to find competitive rates suited to your preferred travel dates.

🔗 URL Citation: [CHEAP FLIGHTS FROM Nice, France to Paris, France (with Prices) [NCE ...](https://www.tripadvisor.com/Flights-g187147-o187234-Nice_t

In [20]:
prompt = "Find some Air France flights from Nice to Paris"

In [21]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_v6ZgLfuNcKzxfyZsV49jIHsj
💬 [2] Created message | ID: msg_NyKOpGeamPUYOxCcb3twrVix
🏃 [3] Created run | ID: run_5UjBh1Hl9EPTE1HKOuhNlXRY
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → Find some Air France flights from Nice to Paris

💡 → You can book Air France flights from Nice to Paris with fares starting at approximately €132 for a round trip. Tickets are available on the Air France website, and flights are frequent between these destinations【3:0†source】. Let me know if you want assistance with specific dates or more details!

🔗 URL Citation: [Vol pas cher Nice Paris (NCE - PAR) | Air France - France](https://wwws.airfrance.fr/fr-fr/vols-de-nice-a-paris)



In [22]:
prompt = "How to book a visit to the Eiffel tower?"

In [23]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_bThOXvQdgS8zsvYxPVswH08u
💬 [2] Created message | ID: msg_WlWEYaXlMMHnG8KdpWfl1Lx9
🏃 [3] Created run | ID: run_c9Q6zbmzlzazgjBaxYcF41hU
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → How to book a visit to the Eiffel tower?

💡 → To book a visit to the Eiffel Tower, follow these steps:

1. **Online Reservation**: Visit the Eiffel Tower's official website or third-party websites like GetYourGuide or TripAdvisor. Search for available tickets, as direct pre-reserved tickets avoid long queues.
   
2. **Choose Your Ticket Type**: Options include access to the second floor, summit, or guided tours. You can also find packages including complimentary activities like Seine river cruises.

3. **Payment and Confirmation**: Complete your payment online. You’ll receive an email confirmation with your ticket.

4. **Visit**: Bring your ticket for access

## Post processing

In [24]:
print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, \nInstructions: {agent.instructions}")

Agent ID: asst_oNmjwUSIbTIvrXMrdmFdkHPu, Name: travelplanneragent, Model: gpt-4o, 
Instructions: You are a travel planner


In [25]:
print("Deleting the agent we have created...")
project_client.delete_agent(agent.id)
print(f"Deleted agent, agent ID: {agent.id}")
print("Done")

Deleting the agent we have created...
Deleted agent, agent ID: asst_oNmjwUSIbTIvrXMrdmFdkHPu
Done
